In [119]:
import pandas as pd
import regex as re
import numpy as np

In [47]:
groups_22 = pd.read_csv('2022_world_cup_groups.csv')

In [48]:
matches_22 = pd.read_csv('2022_world_cup_matches.csv')

In [49]:
data_dict = pd.read_csv('data_dictionary.csv') ## -> fields description

In [134]:
international_matches = pd.read_csv('international_matches.csv')
international_matches.drop(columns={'ID'}, inplace= True)
international_matches.isnull().sum() #Win conditions have null values when matches have ended normally
international_matches['Win Conditions'] = international_matches['Win Conditions'].fillna('normal ending')

In [135]:
international_matches.rename(columns={"Tournament":"Stage","Home Stadium":"Host Team"},inplace = True)

In [136]:
col_list = list(international_matches)

In [137]:
international_matches = international_matches.reindex(columns=['Date','Stage','Home Team','Home Goals',
                                                               'Away Goals','Away Team','Win Conditions','Host Team'])

In [138]:
world_cup_matches = pd.read_csv('world_cup_matches.csv')
world_cup_matches.drop(columns={'ID'}, inplace= True)
world_cup_matches.isnull().sum() #Win conditions have null values when matches have ended normally
world_cup_matches['Win Conditions'] = world_cup_matches['Win Conditions'].fillna('normal ending')

In [139]:
world_cup_matches.drop(columns={'Year'},inplace=True)

## Football History of Spain

In [140]:
spain_international_matches = international_matches[(international_matches['Home Team']=='Spain') | (international_matches['Away Team']=='Spain')]

In [141]:
def find_winner(row):
    # This functions will return if Spain either won or lost a match 
    if row['Home Team']=='Spain' and(row['Home Goals'] > row['Away Goals']):
        return 'Win'
    elif row['Away Team'] == 'Spain' and (row['Home Goals'] < row['Away Goals']):
        return 'Win'
    elif row['Win Conditions'] == 'Spain win on penalties':
        return 'Win'
    else:
        return 'Loss'

In [142]:
spain_international_matches['Result'] = spain_international_matches.apply(lambda row: find_winner(row), axis=1)

C:\Users\juanp\AppData\Local\Temp\ipykernel_19784\3073436608.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spain_international_matches['Result'] = spain_international_matches.apply(lambda row: find_winner(row), axis=1)


In [143]:
spain_world_cup_matches = world_cup_matches[(world_cup_matches['Home Team']=='Spain') | (world_cup_matches['Away Team']=='Spain')]

In [144]:
spain_world_cup_matches['Result'] =  spain_world_cup_matches.apply(lambda row: find_winner(row), axis=1)

C:\Users\juanp\AppData\Local\Temp\ipykernel_19784\2650079529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spain_world_cup_matches['Result'] =  spain_world_cup_matches.apply(lambda row: find_winner(row), axis=1)


In [145]:
spain_history_matches = pd.concat([spain_international_matches, spain_world_cup_matches], ignore_index=True)

In [146]:
spain_history_matches['Win Conditions'].loc[spain_history_matches["Win Conditions"].str.contains("penalties")]="penalties"
# The Win Condition for penalties includes the winner, but to simplify it I'm going to replace with just penalties 

C:\Users\juanp\AppData\Local\Temp\ipykernel_19784\3237819519.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spain_history_matches['Win Conditions'].loc[spain_history_matches["Win Conditions"].str.contains("penalties")]="penalties"


In [147]:
spain_history_matches.to_excel('C:\\Users\\juanp\\proyectos_varios\\world_cup\\spain_history_matches.xlsx')

In [148]:
def goals_favor(row):
    if row['Home Team'] == 'Spain':
        return row['Home Goals']
    elif row['Away Team'] == 'Spain':
        return row['Away Goals']

In [149]:
def goals_against(row):
    if row['Home Team'] != 'Spain':
        return row['Home Goals']
    elif row['Away Team'] != 'Spain':
        return row['Away Goals']

In [150]:
conditions = [
    (spain_history_matches['Home Team'] != 'Spain'),
    (spain_history_matches['Away Team'] != 'Spain')
]

In [151]:
values = [spain_history_matches['Home Team'],spain_history_matches['Away Team'] ]

In [152]:
spain_history_matches['against_team'] = np.select(conditions, values)

In [153]:
spain_history_matches['against_goals'] = spain_history_matches.apply(lambda row: goals_against(row), axis =1)

In [154]:
spain_history_matches['favor_goals'] = spain_history_matches.apply(lambda row: goals_favor(row), axis =1)

In [155]:
spain_history_matches.drop(columns={'Home Team','Home Goals','Away Goals','Away Team'},inplace = True)

In [156]:
list(spain_history_matches)

['Date',
 'Stage',
 'Win Conditions',
 'Host Team',
 'Result',
 'against_team',
 'against_goals',
 'favor_goals']

In [132]:
spain_history_matches= spain_history_matches.reindex(columns=['Date','Stage','Win Conditions','Host Team',
                                       'against_team' 'Result', 'favor_goals', 'agains_goals'])